In [1]:
#Se importan las bibliotecas de arango y pandas 
#ArangoClient es la clase que permite conectar a la instancia desde python
#Pandas es una biblioteca de python para la manipulacion y el análisis de datos.
from arango import ArangoClient
import pandas as pd 

In [10]:
# Importamos la clase ArangoClient del módulo arango.
client = ArangoClient(hosts="http://0.0.0.0:8529/")

# Nos conectamos a la base de datos "AeropuertoDB" con el usuario "root" y su contraseña.
db = client.db("AeropuertoDB", username="root", password="7ey7sjN4loZXmxYS")

In [347]:
# Importamos los datos de los archivos CSV a DataFrames de pandas.

aeropuerto_info = pd.read_csv("DataFrames/Info_Basica_Aeropuertos.csv").iloc[:,1:]
arrivos = pd.read_csv("DataFrames/arrivos.csv").iloc[:,1:]
aerolineas = pd.read_csv("DataFrames/Aerolineas.csv").iloc[:,1:]

# Realizamos algunas operaciones de limpieza y transformación en los datos.
aerolineas.fillna("NO",inplace=True)
aerolineas.drop_duplicates(subset=['IATA'],inplace=True)
aerolineas = aerolineas[aerolineas['IATA'].str.len() <= 2]
arrivos = arrivos.applymap(str)
arrivos.drop_duplicates(inplace=True,subset=['Numero de Vuelo'])
arrivos["cod origen"] = arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)
arrivos['cod destino'] = arrivos.apply(lambda x: x['Destino'].split()[-1].replace('(','').replace(')',''), axis=1)
aeropuerto_info['cod aero'] = aeropuerto_info.apply(lambda x: x['Nombre'].split()[-1].replace('(','').replace(')',''), axis=1)
arrivos['cod vuelo'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2]+a[3:])
arrivos['IATA'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2])

In [348]:
aeropuerto_info.head()

,Nombre,Ubicación,Timezone,Teléfono,cod aero
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD
1,San Salvador Cuscatlan Airport (SAL),"RN-5, San Salvador, El Salvador",(UTC/GMT -6),+503 2349 9455,SAL
2,Panama City Albrook Marcos A. Gelabert Airport...,"Avenida Canfield, Panamá, Panama",(UTC/GMT -5),+507 501-9272,PAC
3,Guatemala City La Aurora Airport (GUA),"Guatemala City, Guatemala",(UTC/GMT -6),+502 2321 5000,GUA
4,Puerto Rico Airport (PUR),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,PUR


In [351]:
aeropuerto_info_db = db.create_collection("aeropuerto")
arrivos_db = db.create_collection("arrivos")
aerolineas_db = db.create_collection("aerolineas")

In [352]:
aeropuerto_info_db.add_hash_index(fields=["Nombre"], unique=True)

{'id': '107150',
 'fields': ['Nombre'],
 'type': 'hash',
 'name': 'idx_1780077693720592384',
 'deduplicate': True,
 'sparse': False,
 'unique': True,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [353]:
for row in range(aeropuerto_info.shape[0]):
        
    aeropuerto_info_db.insert({'_key':aeropuerto_info.iloc[row,0][-4:-1],
                               aeropuerto_info.columns[0]:aeropuerto_info.iloc[row,0],
                               aeropuerto_info.columns[1]:aeropuerto_info.iloc[row,1],
                               aeropuerto_info.columns[2]:aeropuerto_info.iloc[row,2],
                               aeropuerto_info.columns[3]:aeropuerto_info.iloc[row,3]})

In [355]:
#Insertar base de datos de arrivos

arrivos.head()

,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo,IATA
0,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,SDQ,5X402,5X
1,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,SDQ,AV208,AV
2,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,SDQ,CM107,CM
3,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,SDQ,TK9551,TK
4,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,SDQ,KL3056,KL


In [354]:
arrivos_db.add_hash_index(fields=["Numero de Vuelo"], unique=False)

{'id': '107191',
 'fields': ['Numero de Vuelo'],
 'type': 'hash',
 'name': 'idx_1780077714756075520',
 'deduplicate': True,
 'sparse': False,
 'unique': False,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [357]:
for row in range(arrivos.shape[0]):
    
    arrivos_db.insert({ '_key': arrivos.iloc[row,2][0:2]+arrivos.iloc[row,2][3:],
                       arrivos.columns[2]:arrivos.iloc[row,2],
                       arrivos.columns[3]:arrivos.iloc[row,3],
                       arrivos.columns[4]:arrivos.iloc[row,4]
                      })

In [358]:
#Insertar base de datos de aerolineas

aerolineas.head()

,IATA,IACO,Aerolinea,Llamada de Identificación,País
0,PR,BOI,2GO,ABAIR,Philippines
1,1A,NO,Amadeus IT Group S.A.,NO,Global
2,1B,NO,Sabre travel network Asia-Pacific (ex-Abacus),NO,APAC
3,1E,NO,Travelsky,NO,China
4,1F,NO,"Infini travel information, Inc.",NO,Japan


In [359]:
aerolineas_db.add_hash_index(fields=["IATA"], unique=True)

{'id': '117368',
 'fields': ['IATA'],
 'type': 'hash',
 'name': 'idx_1780077941978300416',
 'deduplicate': True,
 'sparse': False,
 'unique': True,
 'selectivity': 1,
 'new': True,
 'cacheEnabled': False,
 'estimates': True}

In [360]:
for row in range(aerolineas.shape[0]):
    
    aerolineas_db.insert({'_key':aerolineas.iloc[row,0].lower(),
                          aerolineas.columns[0]:aerolineas.iloc[row,0],
                          aerolineas.columns[1]:aerolineas.iloc[row,1],
                          aerolineas.columns[2]:aerolineas.iloc[row,2],
                          aerolineas.columns[3]:aerolineas.iloc[row,3],
                          aerolineas.columns[4]:aerolineas.iloc[row,4]
                      })

In [389]:
# Crear gráfico
ae = db.create_graph('Aeropuerto')

In [390]:
#Ahora hacemos un inner join para saber cuales son los viajes que tienen como origen los aeropuertos de la base.

arrivos_dest = arrivos.rename(columns={"cod origen":'cod aero'})
s1 = pd.merge(aeropuerto_info, arrivos_dest, how='inner', on=['cod aero'])
s1.head()

,Nombre,Ubicación,Timezone,Teléfono,cod aero,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod destino,cod vuelo,IATA
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,UA 1285,7:40,10:00,IAH,UA1285,UA
1,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,AV 2312,7:40,10:00,IAH,AV2312,AV
2,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,NZ 6850,7:40,10:00,IAH,NZ6850,NZ
3,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,UA 1863,9:19,nan,IAH,UA1863,UA
4,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD,Chicago [ORD],Houston George Bush Intercontinental Airport (...,NZ 6856,9:19,nan,IAH,NZ6856,NZ


In [391]:
#Insertamos las relaciones de cada nódulo, aquí es de los aeropuerto de origen a número de vuelo

aero_viajes = ae.create_edge_definition(
    edge_collection='salida',
    from_vertex_collections=['aeropuerto'],
    to_vertex_collections=['arrivos'])


for i in range(len(s1)):
    aero_viajes.insert({
        '_key': f'{s1["cod aero"][i]}-{s1["cod vuelo"][i]}',
        '_from': f'aeropuerto/{s1["cod aero"][i]}',
        '_to': f'arrivos/{s1["cod vuelo"][i]}'
    })
    aero_viajes.edges(f'{s1["cod aero"][i]}-{s1["cod vuelo"][i]}', direction='out')
    #Se hace la dirección para afuera, para que desde el aeropuerto de salida/origen tenga la flecha hacia
    # el número de vuelo.


In [392]:
#Ahora hacemos un inner join para saber cuales son los viajes que tienen como destino los aeropuertos de la base.

arrivos_dest = arrivos.rename(columns={"cod destino":'cod aero'})
s2 = pd.merge(aeropuerto_info, arrivos_dest, how='inner', on=['cod aero'])
s2.head()

,Nombre,Ubicación,Timezone,Teléfono,cod aero,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod vuelo,IATA
0,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ,Miami [MIA],Santo Domingo Las Americas Airport (SDQ),5X 402,8:40,10:38,MIA,5X402,5X
1,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ,Bogota [BOG],Santo Domingo Las Americas Airport (SDQ),AV 208,7:10,10:55,BOG,AV208,AV
2,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),CM 107,21:41,01:16,PTY,CM107,CM
3,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),TK 9551,21:41,01:16,PTY,TK9551,TK
4,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ,Panama City [PTY],Santo Domingo Las Americas Airport (SDQ),KL 3056,21:41,01:16,PTY,KL3056,KL


In [393]:
#Insertamos las relaciones de cada nódulo, aquí es de los viajes a aeropuertos de destino
aero_viajes_arrivo = ae.create_edge_definition(
    edge_collection='Destino',
    from_vertex_collections=['arrivos'],
    to_vertex_collections=['aeropuertos'])

for i in range(len(s2)):
    aero_viajes_arrivo.insert({
        '_key': f'{s2["cod aero"][i]}--{s2["cod vuelo"][i]}',
        '_from': f'arrivos/{s2["cod vuelo"][i]}',
        '_to': f'aeropuerto/{s2["cod aero"][i]}'
    })
    aero_viajes_arrivo.edges(f'{s2["cod aero"][i]}--{s2["cod vuelo"][i]}', direction='out') 
    #Se hace la dirección para afuera, esto pues queremos que la flecha vaya para el aeropuerto de destino. 

In [394]:
#Ahora hacemos un inner join para saber cuales son los viajes que pertenecen a las aerolineas

s3 = pd.merge(aerolineas, arrivos, how='inner', on=['IATA'])
s3.head()

,IATA,IACO,Aerolinea,Llamada de Identificación,País,Origen,Destino,Numero de Vuelo,Salida,Llegada,cod origen,cod destino,cod vuelo
0,PR,BOI,2GO,ABAIR,Philippines,Manila [MNL],San Francisco Airport (SFO),PR 104,2:00,nan,MNL,SFO,PR104
1,4Y,OCN,Eurowings Discover,EUROWINGS DISCOVER,Germany,Frankfurt [FRA],Orlando Airport (MCO),4Y 68,14:19,18:09,FRA,MCO,4Y68
2,6E,IGO,IndiGo,IFLY,India,Istanbul [IST],New York John F. Kennedy Airport (JFK),6E 4235,20:32,23:46,IST,JFK,6E4235
3,6E,IGO,IndiGo,IFLY,India,Istanbul [IST],San Francisco Airport (SFO),6E 4272,8:25,12:29,IST,SFO,6E4272
4,GB,ABX,ABX Air,ABEX,United States,Covington [CVG],Houston George Bush Intercontinental Airport (...,GB 2024,5:30,07:13,CVG,IAH,GB2024


In [395]:
#Insertamos las relaciones de cada nódulo, aquí es del número de avión con la aerolinea a la cúal pertenece

aerolineas_viajes = ae.create_edge_definition(
    edge_collection='aa',
    from_vertex_collections=['arrivos'],
    to_vertex_collections=['aerolineas'])

for i in range(len(s1)):
    aerolineas_viajes.insert({
        '_key': f'{s3["IATA"][i]}-{s3["cod vuelo"][i]}',
        '_from': f'arrivos/{s3["cod vuelo"][i]}',
        '_to': f'aerolineas/{s3["IATA"][i].lower()}'
    })